In [1]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=be483bd5dfcfa5d2075202124ba0551512f1e853eccf0ef6935161749b0d0335
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=51acf5a6fb0fb120a204e032c9617037c14bfb0567ab9eeef6e2edce1f006002
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [2]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 75.2 MB/s eta 0:00:00


In [1]:
import logging
logging.getLogger("pytorchvideo").setLevel(logging.CRITICAL)

import torch
from torch import nn
import numpy as np
from tqdm import tqdm
import os
import torchvision.utils as vutils

# X3D 모델 로드
model_name = 'x3d_s'
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
device = "cuda"
feature_model = model.eval().to(device)
del feature_model.blocks[-1]

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/X3D_S.pyth" to /root/.cache/torch/hub/checkpoints/X3D_S.pyth
100%|██████████| 29.4M/29.4M [00:02<00:00, 13.3MB/s]


In [2]:
def load_deweather_model(device):
    from drive.MyDrive.pipeline.model.ESDNet import ESDNet
    deweather_model = ESDNet(
        en_feature_num=48,
        en_inter_num=32,
        de_feature_num=64,
        de_inter_num=32,
        sam_number=1
    ).to(device)

    load_path = '/content/drive/MyDrive/pipeline//weight/deweathering_model.pth'
    try:
        ckpt = torch.load(load_path, map_location=device)
        state_dict = ckpt if load_path.endswith('.pth') else ckpt['state_dict']
        deweather_model.load_state_dict(state_dict)
        print("Deweathering model loaded from", load_path)
    except Exception as e:
        print("Deweathering model checkpoint 로드 실패:", e)
    deweather_model.eval()
    return deweather_model

In [3]:
deweather_model = load_deweather_model(device).to(device)

Deweathering model loaded from /content/drive/MyDrive/pipeline//weight/deweathering_model.pth


In [7]:
!pip install performer_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 115.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [4]:
from drive.MyDrive.pipeline.model.classifier import Model

classifier = Model(ff_mult=1, dims=(32, 32), depths=(1, 1))
ckpt_path = "/content/drive/MyDrive/pipeline/weight/De_final_model.pth"
classifier.load_state_dict(torch.load(ckpt_path, map_location=device))
classifier.to(device)

Model(
  (stages): ModuleList(
    (0): ConvBlock(
      (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (conv): DECOUPLED(
        (norm2d): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm1d): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2d): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
        (conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), groups=16)
      )
      (ff): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=32, out_features=32, bias=True)
        (4): GELU(approximate='none')
      )
    )
    (1): Sequential(
      (0): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=

In [14]:
import cv2
import torch
import numpy as np
from torchvision.transforms import ToTensor, Normalize, Resize, Compose
from collections import deque
from PIL import Image
import time
from google.colab.patches import cv2_imshow

# ======================
# 설정
# ======================
CLIP_LEN = 13
IMG_SIZE = 160
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MEAN = [0.45, 0.45, 0.45]
STD = [0.225, 0.225, 0.225]
MAX_FRAMES = 200
ANOMALY_THRESHOLD = 0.60
ALERT_FRAMES = 20

# ======================
# 전처리 정의
# ======================
transform_frame = Compose([
    Resize((IMG_SIZE, IMG_SIZE)),
    ToTensor()
])
normalize = Normalize(mean=MEAN, std=STD)
frame_buffer = deque(maxlen=CLIP_LEN)
anomaly_streak = 0  # 연속 이상 프레임 수

# ======================
# 비디오 캡처
# ======================
cap = cv2.VideoCapture("/content/drive/MyDrive/pipeline/UCF_synth/UCF_Crimes/Videos/Burglary/Burglary001_x264.mp4")
assert cap.isOpened(), "❌ 카메라 또는 영상 열기 실패"

frame_count = 0
start_time = time.time()

while frame_count < MAX_FRAMES:
    ret, frame = cap.read()
    if not ret:
        print("📉 영상 종료 또는 오류")
        break

    # 1. OpenCV → PIL → Tensor
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    frame_tensor = transform_frame(frame_pil).unsqueeze(0).to(DEVICE)  # [1, 3, H, W]

    # 2. Deweathering
    with torch.no_grad():
        output, _, _ = deweather_model(frame_tensor)
        clean_frame = output[0] if isinstance(output, tuple) else output  # [1, 3, H, W]

    # 3. clean_frame: 시각화를 위한 변환
    clean_img_np = clean_frame.squeeze(0).detach().cpu().numpy()  # [3, H, W]
    clean_img_np = np.transpose(clean_img_np, (1, 2, 0))  # [H, W, C]
    clean_img_np = np.clip(clean_img_np * 255.0, 0, 255).astype(np.uint8)  # RGB
    clean_img_bgr = cv2.cvtColor(clean_img_np, cv2.COLOR_RGB2BGR)

    # 4. 버퍼에 추가
    frame_buffer.append(clean_frame.squeeze(0))  # [3, H, W]

    # 5. 클립이 모이면 처리
    label = "Collecting..."
    anomaly_score = None
    if len(frame_buffer) == CLIP_LEN:
        clip_tensor = torch.stack(list(frame_buffer), dim=0).permute(1, 0, 2, 3)  # [3, T, H, W]
        normalized_clip = torch.stack([
            normalize(clip_tensor[:, t, :, :]) for t in range(CLIP_LEN)
        ], dim=1)  # [3, T, H, W]
        clip_tensor = normalized_clip.unsqueeze(0).to(DEVICE)  # [1, 3, T, H, W]

        with torch.no_grad():
            feat = feature_model(clip_tensor)
            pred, _ = classifier(feat)
            anomaly_score = torch.sigmoid(pred).item()
            label = f"A: {anomaly_score:.2f}"

        # 이상 상태 연속 체크
        if anomaly_score >= ANOMALY_THRESHOLD:
            anomaly_streak += 1
        else:
            anomaly_streak = 0

    # 6. FPS 계산
    elapsed = time.time() - start_time
    fps = frame_count / elapsed if elapsed > 0 else 0
    fps_label = f"FPS: {fps:.2f}"

    # 7. 텍스트 표시 (좌측 상단)
    text_position = (10, 20)
    cv2.putText(clean_img_bgr, fps_label + " | " + label, text_position,
                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 255), 1)


    # 8. 테두리 추가 조건
    if anomaly_streak >= ALERT_FRAMES:
        h, w = clean_img_bgr.shape[:2]
        cv2.rectangle(clean_img_bgr, (0, 0), (w - 1, h - 1), (0, 0, 255), 3)

    # 9. 출력
    cv2_imshow(clean_img_bgr)

    frame_count += 1

cap.release()
print("✅ 처리 완료")

Output hidden; open in https://colab.research.google.com to view.